In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score

In [5]:
milk = pd.read_csv('milk.csv',index_col=0)

In [6]:
milk.head()

,water,protein,fat,lactose,ash
Animal,,,,,
HORSE,90.1,2.6,1.0,6.9,0.35
ORANGUTAN,88.5,1.4,3.5,6.0,0.24
MONKEY,88.4,2.2,2.7,6.4,0.18
DONKEY,90.3,1.7,1.4,6.2,0.40
HIPPO,90.4,0.6,4.5,4.4,0.10


In [19]:
scaler = StandardScaler().set_output(transform='pandas')
milk_scaled = scaler.fit_transform(milk)
clust_DB = DBSCAN(eps=1,min_samples=2)
clust_DB.fit(milk_scaled)
silhouette_score(milk_scaled,clust_DB.labels_)

0.2676764654728567

In [20]:
eps_range = [0.2,0.4,0.6,1]
mp_range = [2,3,4,5]
cnt = 0
a =[]
for i in eps_range:
    for j in mp_range:
        clust_DB = DBSCAN(eps=i, min_samples=j)
        clust_DB.fit(milk_scaled.iloc[:,:5])
        if len(set(clust_DB.labels_)) > 2:
            cnt = cnt + 1
            milk_scaled['Clust'] = clust_DB.labels_
            milk_scl_inliers = milk_scaled[milk_scaled['Clust']!=-1]
            sil_sc = silhouette_score(milk_scl_inliers.iloc[:,:-1],
                             milk_scl_inliers.iloc[:,-1])
            a.append([cnt,i,j,sil_sc])
            print(i,j,sil_sc)
    
a = np.array(a)
pa = pd.DataFrame(a,columns=['Sr','eps','min_pt','sil'])
print("Best Paramters:")
pa[pa['sil'] == pa['sil'].max()]


0.4 2 0.6518937593821538
0.4 3 0.5385180352469559
0.6 2 0.5934459505692155
0.6 3 0.5344431042454363
0.6 4 0.5519747727201489
1 2 0.4344818095328392
1 3 0.6473871775367226
Best Paramters:


,Sr,eps,min_pt,sil
0,1.0,0.4,2.0,0.651894


#### DBSCAN with best params

In [47]:
clust_DB = DBSCAN(eps=0.4,min_samples=2)
clust_DB.fit(milk_scaled.iloc[:,:5])


DBSCAN(eps=0.4, min_samples=2)

## Cluster analysis

In [56]:
milk_clust = milk.copy()
milk_clust['cluster'] = clust_DB.labels_
milk_clust = milk_clust[milk_clust['cluster']!=-1]
milk_clust.groupby('cluster').mean()

,water,protein,fat,lactose,ash
cluster,,,,,
0,88.450,1.800000,3.100000,6.200000,0.210000
1,86.825,3.800000,3.275000,5.350000,0.777500
2,81.900,6.033333,6.733333,4.766667,0.873333
3,65.350,10.550000,20.000000,2.550000,1.400000


In [57]:
milk_clust.sort_values('cluster')

,water,protein,fat,lactose,ash,cluster
Animal,,,,,,
ORANGUTAN,88.5,1.4,3.5,6.0,0.24,0
MONKEY,88.4,2.2,2.7,6.4,0.18,0
CAMEL,87.7,3.5,3.4,4.8,0.71,1
BISON,86.9,4.8,1.7,5.7,0.90,1
LLAMA,86.5,3.9,3.2,5.6,0.80,1
ZEBRA,86.2,3.0,4.8,5.3,0.70,1
BUFFALO,82.1,5.9,7.9,4.7,0.78,2
FOX,81.6,6.6,5.9,4.9,0.93,2
SHEEP,82.0,5.6,6.4,4.7,0.91,2
